# Resolution: Household 4428 Has Multiple Tax Units

The apparent bug is explained by household 4428 having 2 tax units with different itemization decisions.

In [ ]:
from policyengine_us import Microsimulation
from policyengine_us.model_api import *
import numpy as np

DATASET = "hf://policyengine/policyengine-us-data/enhanced_cps_2024.h5"
YEAR = 2026

## Setup Reform

In [ ]:
reform = Reform.from_dict({
    "gov.irs.income.bracket.rates.2": {"2026-01-01": 0.15},
    "gov.irs.income.bracket.rates.3": {"2026-01-01": 0.25},
    "gov.irs.deductions.standard.amount.JOINT": {"2026-01-01": 48900},
    "gov.irs.income.exemption.amount": {"2026-01-01": 0},
    "gov.contrib.reconciliation.ctc.in_effect": {"2026-01-01": True},
    "gov.irs.credits.ctc.amount.base[0].amount": {"2026-01-01": 2200},
}, country_id="us")

sim = Microsimulation(reform=reform, dataset=DATASET)

## Check Both Tax Units

In [ ]:
# Get tax unit data
tax_unit_ids = sim.calculate("tax_unit_id", period=YEAR).values
itemizes = sim.calculate("tax_unit_itemizes", period=YEAR).values
tax_if_item = sim.calculate("tax_liability_if_itemizing", period=YEAR).values
tax_if_std = sim.calculate("tax_liability_if_not_itemizing", period=YEAR).values

# Find the two tax units in household 4428
tu1_idx = np.where(tax_unit_ids == 442801)[0][0]
tu2_idx = np.where(tax_unit_ids == 442802)[0][0]

print("Tax Unit 442801 (parents + 4 kids):")
print(f"  Itemizes: {bool(itemizes[tu1_idx])}")
print(f"  Tax if itemizing: ${tax_if_item[tu1_idx]:,.2f}")
print(f"  Tax if standard: ${tax_if_std[tu1_idx]:,.2f}")
print(f"  Optimal: {'Same either way' if tax_if_item[tu1_idx] == tax_if_std[tu1_idx] else 'Itemize' if tax_if_item[tu1_idx] < tax_if_std[tu1_idx] else 'Standard'}")

print("\nTax Unit 442802 (23-year-old):")
print(f"  Itemizes: {bool(itemizes[tu2_idx])}")
print(f"  Tax if itemizing: ${tax_if_item[tu2_idx]:,.2f}")
print(f"  Tax if standard: ${tax_if_std[tu2_idx]:,.2f}")
print(f"  Optimal: {'Standard' if tax_if_std[tu2_idx] < tax_if_item[tu2_idx] else 'Itemize'}")

print("\n✓ Both tax units make optimal decisions!")

## The Real Issue

The confusion arose from looking at household-level aggregates. When mapped to household level:
- `tax_unit_itemizes` shows True (from the first tax unit)
- But the tax liability values are summed across both tax units

This creates the appearance of irrational behavior when it's actually two rational decisions.